In [2]:
!pip install google-auth google-auth-httplib2 google-auth-oauthlib google-api-python-client
!pip install -q -U google-generativeai
!pip install --upgrade google-generativeai

In [ ]:
#Import necessary libraries
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [ ]:
import openpyxl
from openai import OpenAI
import os
import json
from tqdm import tqdm
import pandas as pd
import google.generativeai as genai
from google.generativeai.types import RequestOptions
from google.api_core import retry
import time

In [ ]:
# Import the API Key (insert key here)
GOOGLE_API_KEY=''
genai.configure(api_key=GOOGLE_API_KEY)

# Reading G-Drive Sheet

In [ ]:
!pip install --quiet gspread oauth2client google-auth
import gspread
from google.colab import auth
import pandas as pd
import google.auth

auth.authenticate_user()

# Authorize gspread client
# Use google.auth.default() to get credentials
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

def load_sheet(sheet_id, sheet_name="Sheet1"):
    # Load google sheet as dataframe
    worksheet = gc.open_by_key(sheet_id).worksheet(sheet_name)
    data = worksheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

path = "1NHZZ5YklIMuMicMwZghzKfmLGBTVDI6lzRVIJyqVrU4"
sheet = load_sheet(path,"Determining What Students Know Explain_responses")  # There is only one sheet, so no ambiguity here.

Get all the responses and scores

In [ ]:
INPUT_COLUMN = "response" # Specify the column from which input is read (A->1, B->2, etc.)
row_count = len(sheet) # Get number of rows. If number of rows to read is already known (or sheet does not terminate where data terminates), place number here (+1) instead.inputs = [sheet.loc[i, INPUT_COLUMN]  for i in range(2, row_count)]  # Column number is set here. YOUR INPUT COLUMN NUMBER GOES HERE.
inputs = [sheet.loc[i, INPUT_COLUMN] for i in range(row_count)]
inputs = [x for x in inputs if x is not None]
print(inputs)
print(len(inputs))

# Gemini Prompt Setup

Scoring Prompt Input

In [ ]:
GEMINI_SYSTEM_PROMPT = """
You are a tutor evaluator. Please score the following tutor response to a tutor training scenario involving a middle school student as follows:
-If the tutor's response demonstrates that the tutor understands the importance of students explaining what they already know or the importance of asking students open-ended questions to assess their prior knowledge, score it with a 1. Sample responses scoring a 1 include: “because it would help me assess his prior knowledge about the order of operations”; “It acknowledges the work done and elicits an answer on student's understanding of next step”; “Introducing terminology he isn't familiar with might be confusing, so I would only use the term 'order of operations' if he introduced it. I also wouldn't give him the next step, I would see if he could figure out what it could be. Asking what he could do next gives a sense of what he knows and I can ask follow up questions to gain more insight”; “Asking them what they think should happen next allows them to tell you what they understand about the question coming into it”; “It engages him and let’s him build on things that he already knows to solve problems.”
-If the tutor's response does not demonstrate that the tutor understands the importance of asking open questions and assessing student’s prior knowledge or what they already know, score it with a 0. Sample responses scoring a 0 include: “Roberto is struggling, but has some basic knowledge of the subject. He needs to be encouraged so that his self confidence can improve. He will strive to solve the problem”; “He looks like he is on the correct path”; “provides context for solving the problem, positive feedback, and encourages further work”; “It will encourage Roberto to keep on trying”; “Because it's helpful to acknowledge what is correct so far so the student can move forward.”
Once given a response by the user, please return a JSON string following the format, {"Rationale": "your reasoning here", "Score":0/1}
"""

Helper function for response parsing

In [ ]:
def extract_response(response_obj, json=False):
  role = response_obj.choices[0].message.role
  content = response_obj.choices[0].message.content
  if json:
    return {"role": role, "content": content}
  else:
    return (role, content)

## gemini API Call

In [ ]:
# Iterate over all responses
MAX_TOKENS = 300
TEMPERATURE = 0
RUN_UP_TO = 256  # Sets a maximum index for responses to run. Useful to specify how many responses we want to run on (partial execution). Set to -1 to run them all.
TWO_STAGE = False  # Specifies whether to refine the feedback provided by the scoring prompt
TWO_STAGE_INCLUDE_RESPONSE = False # Specifies whether the second-stage prompt uses the original response.
SCORE_COLUMN = "response"  # Change column numbers here to  modify where output is written


MODEL = 'gemini-2.5-pro'
model = genai.GenerativeModel(MODEL, system_instruction=GEMINI_SYSTEM_PROMPT)

if RUN_UP_TO >=  0:  # If an upper bound is set
  inputs_upto = inputs[:RUN_UP_TO]
else:
  inputs_upto = inputs  # Take the whole set of responses
tempScoreList = []
tempDirectionList = []
tempRationaleList = []
retries = []

for index, inpt in tqdm(enumerate(inputs_upto), total=len(inputs_upto)):
  if index%60 == 0:
    time.sleep(30)
  generation_prompt = "Tutor Response: " + inpt + "\n\n. Your JSON: " # Gemini Change
  generation_config = genai.GenerationConfig(temperature=TEMPERATURE)
  # Use client.chat.completions.create instead of client.generate_text
  gemini_out = model.generate_content(generation_prompt, generation_config=generation_config,request_options=RequestOptions(retry=retry.Retry(initial=1.0,multiplier=2.0,maximum=60.0))) # TODO: Potential break point
  # Extract the content from the response
  content = gemini_out.text.lstrip("```json")[:-4]
  try:
    content_json = json.loads(content)  # Run response through JSON
    score = str(content_json["Score"])  # Cast to string to avoid type inequality
    rationale = str(content_json["Rationale"])  # Fetch the rationale
    sheet.at[index,"GPT Score"] = score  # Now write both into the dataframe
    sheet.at[index,"GPT Rationale"] = rationale
    tempScoreList.append(score)
    tempDirectionList.append(generation_prompt)
    tempRationaleList.append(rationale)
    #rationale_cell.value = rationale

  except:
    print("error!")
    tempScoreList.append("error during LLM evaluation")
    tempRationaleList.append("error during LLM evaluation")
    retries.append((index,inpt))
  #   score_cell.value = "---"
  #   rationale_cell.value = "---"  # Failsafe
  #   if TWO_STAGE:
  #     refined_feedback_cell.value = "---"
print(tempScoreList)
print(tempDirectionList)
print(tempRationaleList)

In [ ]:
print(tempScoreList)
print(tempRationaleList)

In [ ]:
for i in range(len(inputs_upto)):
  print(inputs_upto[i])
  print(tempScoreList[i])
  print(tempRationaleList[i])
  print()

In [ ]:
print(list(sheet["GPT Score"]))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Save G-Drive Sheet

In [ ]:
sheet.to_csv('/content/drive/Shareddrives/PLUS/Research/Clara Stuff/Determining_What_Students_Know_Gemini_Explain_Eval.csv', index=False)